In [1]:
import torch
import os
import torch.nn as nn
import numpy as np
from torch.utils.data import DataLoader
from network import CharacterClassifier
from tqdm import tqdm
from training_data import CombinedData
from PIL import Image
from matplotlib import pyplot as plt

hasy_train = CombinedData('HASY')
hasy_test = CombinedData('HASY', train=False)

print("Train data length: {0}".format(len(hasy_train.data)))
print("Test data length: {0}".format(len(hasy_test.data)))
print("Img Shape: {0}".format(hasy_train.data[0].shape))
print("Number of Labels: {0}".format(hasy_train.no_labels))

  8%|▊         | 812/10000 [00:00<00:01, 8116.37it/s]

No training data for 0. Skipping
No training data for 1. Skipping
No training data for 2. Skipping
No training data for 3. Skipping
No training data for 4. Skipping
No training data for 5. Skipping
No training data for 6. Skipping
No training data for 7. Skipping
No training data for 8. Skipping
No training data for 9. Skipping
No training data for +. Skipping


100%|██████████| 10000/10000 [00:00<00:00, 363253.28it/s]

No training data for 0. Skipping
No training data for 1. Skipping
No training data for 2. Skipping
No training data for 3. Skipping
No training data for 4. Skipping
No training data for 5. Skipping
No training data for 6. Skipping
No training data for 7. Skipping
No training data for 8. Skipping
No training data for 9. Skipping
No training data for +. Skipping
Train data length: 60405
Test data length: 10045
Img Shape: torch.Size([1, 32, 32])
Number of Labels: 11


In [3]:
from torchvision import models
from torch.nn import Conv2d

train_loader = DataLoader(hasy_train, batch_size=16, shuffle=True)
test_loader = DataLoader(hasy_test, batch_size=16, shuffle=False)

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")


def calc_accuracy(model):
    accuracies = []
    for idx, [x_test, y_test] in enumerate(test_loader):
        x_test, y_test = x_test.to(device), y_test.to(device)
        test_pred = model(x_test)
        accuracy = 100 * torch.mean((torch.argmax(test_pred, dim=1) == y_test).float())
        accuracies.append(accuracy.item())
    return np.mean(accuracies) 

def print_stats(lr, beta1, beta2, decay, accuracy):
    print("Learning rate: {0}".format(lr))
    print("Beta 1: {0}".format(beta1))
    print("Beta 2: {0}".format(beta2))
    print("Weight decay: {0}".format(decay))
    print("Accuracy: {0}".format(accuracy))

learning_rates = [0.01, 0.001, 0.0001]
betas1 = [0.8, 0.85, 0.9, 0.95]
betas2 = [0.9, 0.925, 0.95, 0.99]
weight_decays = [0, 0.01, 0.001, 0.0001]

best_lr = 0
best_beta1 = 0
best_beta2 = 0
best_decay = 0
best_accuracy = 0

for lr in learning_rates:
    for beta1 in betas1:
        for beta2 in betas2:
            for decay in weight_decays:
                model = models.alexnet(num_classes=11)
                model.features[0] = Conv2d(1, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
                model.to(device)
                optimizer = torch.optim.Adam(model.parameters(), lr=lr, betas=(beta1, beta2), weight_decay=decay)
                criterion = nn.CrossEntropyLoss()
                for step, [x_train, y_train] in enumerate(tqdm(train_loader)):
                    x_train, y_train = x_train.to(device), y_train.to(device)
                    optimizer.zero_grad()
                    train_pred = model(x_train)
                    loss = criterion(train_pred, y_train)
                    loss.backward()
                    optimizer.step()
                accuracy = calc_accuracy(model)
                print_stats(lr, beta1, beta2, decay, accuracy)
                if accuracy > best_accuracy:
                    best_lr = lr
                    best_beta1 = beta1
                    best_beta2 = beta2
                    best_decay = decay
                    best_accuracy = accuracy
                    
print("Best hyperparameters:")
print_stats(best_lr, best_beta1, best_beta2, best_decay, best_accuracy)
                

100%|██████████| 3776/3776 [01:43<00:00, 36.46it/s]


Learning rate: 0.01
Beta 1: 0.8
Beta 2: 0.9
Weight decay: 0
Accuracy: 11.298076923485775


100%|██████████| 3776/3776 [01:55<00:00, 32.78it/s]


Learning rate: 0.01
Beta 1: 0.8
Beta 2: 0.9
Weight decay: 0.01
Accuracy: 10.733096522130785


100%|██████████| 3776/3776 [01:54<00:00, 32.89it/s]


...

Learning rate: 0.0001
Beta 1: 0.8
Beta 2: 0.9
Weight decay: 0
Accuracy: 96.36743630573248


100%|██████████| 3776/3776 [01:55<00:00, 32.63it/s]


Learning rate: 0.0001
Beta 1: 0.8
Beta 2: 0.9
Weight decay: 0.01
Accuracy: 92.8818593966733


 49%|████▉     | 1856/3776 [00:56<00:57, 33.21it/s]

KeyboardInterrupt: 

In [5]:
print("Best hyperparameters:")
print_stats(best_lr, best_beta1, best_beta2, best_decay, best_accuracy)

Best hyperparameters:
Learning rate: 0.001
Beta 1: 0.8
Beta 2: 0.99
Weight decay: 0.001
Accuracy: 97.73089171974522


In [7]:
model = models.alexnet(num_classes=11)
model.features[0] = Conv2d(1, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
model.to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=0.0001, betas=(0.8, 0.99), weight_decay=0.001)
criterion = nn.CrossEntropyLoss()
for step, [x_train, y_train] in enumerate(tqdm(train_loader)):
    x_train, y_train = x_train.to(device), y_train.to(device)
    optimizer.zero_grad()
    train_pred = model(x_train)
    loss = criterion(train_pred, y_train)
    loss.backward()
    optimizer.step()
accuracy = calc_accuracy(model)
print_stats(lr, beta1, beta2, decay, accuracy)
if accuracy > best_accuracy:
    best_lr = lr
    best_beta1 = beta1
    best_beta2 = beta2
    best_decay = decay
    best_accuracy = accuracy


 16%|█▌        | 602/3776 [00:19<01:35, 33.18it/s]


 32%|███▏      | 1202/3776 [00:37<01:22, 31.35it/s]


 47%|████▋     | 1763/3776 [00:55<01:04, 31.44it/s]


 62%|██████▏   | 2355/3776 [01:13<00:43, 32.76it/s]


 78%|███████▊  | 2947/3776 [01:31<00:25, 32.61it/s]


 94%|█████████▎| 3539/3776 [01:50<00:07, 31.62it/s]


100%|██████████| 3776/3776 [01:58<00:00, 31.93it/s]


Learning rate: 0.0001
Beta 1: 0.8
Beta 2: 0.9
Weight decay: 0.001
Accuracy: 96.1484872611465
